<a href="https://colab.research.google.com/github/rubaahmedkhan/AgenticAI-Toolkit-/blob/main/tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⚙️ SDK Kya Karta Hai?
Jab aap koi Python function SDK mein "tool" ke tor par use karte hain, to SDK khud-ba-khud yeh kaam karta hai:

1. Tool ka Naam:
Agar aap naam nahi dete, to function ka naam hi tool ka naam ban jata hai.

python
Copy
Edit
def get_weather(city: str): ...
# Tool name: get_weather
2. Tool ka Description:
Function ke docstring se liya jata hai (ya aap manually de sakte hain).

python
Copy
Edit
def get_weather(city: str):
    """Get the current weather for a given city."""
    ...
# Tool description: "Get the current weather for a given city."
3. Input Schema Automatically Ban Jata Hai:
Python ke function arguments (e.g., city: str) se input ka schema banta hai.

Aapko manually JSON schema banani ki zarurat nahi hoti.

4. Input Argument Descriptions:
Har input argument ka description bhi docstring se nikaala jata hai (agar enable ho).

🔍 Yeh Sab Kaise Hota Hai? (Backend Details)
SDK use karta hai:

inspect module → function signature ko analyze karne ke liye.

griffe → docstring ko parse karne ke liye.

pydantic → input validation aur schema generation ke liye.









In [ ]:
!pip install  -qU openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.6/734.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os

from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, ModelSettings, function_tool
from agents.run import RunConfig
from google.colab import userdata


In [ ]:
gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
model_settings = ModelSettings(
    max_tokens=50,
                                              # CHECK All PARAMETERS PICTURE IN MOBILE
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    model_settings=model_settings,
    tracing_disabled=True
)

In [ ]:
@function_tool
def greet(name: str):
    """Say hello to someone."""
    return f"Hello, {name}!"


In [ ]:
tool = function_tool(greet)


In [ ]:
# Tool NAME OVERWRIGHT

@function_tool(name_override="custom_greeting") #Now tool name will be custom_greeting, not greet.


def greet(name: str):
    """Say hello to someone."""
    return f"Hello, {name}!"


In [ ]:
#🔹3. description_override: str | None = None

# Function ki docstring ke bajaye, aap manually description de sakte hain.

@function_tool(description_override="This tool greets users in a custom way.")
def greet(name: str):
    """Old docstring ignored."""
    return f"Hello, {name}!"


In [ ]:
# 🔹4. docstring_style: DocstringStyle | None = None
# Batata hai ke docstring ka format kaunsa hai — like google, numpy, ya sphinx. SDK griffe library se docstring ko parse karta hai.

@function_tool(docstring_style="google")
def get_sum(a: int, b: int):
    """
    Add two numbers.

    Args:
        a: First number.
        b: Second number.
    """
    return a + b


In [ ]:
# 🔹5. use_docstring_info: bool = True
# ✅ Kya karta hai:
# Agar True ho to function ke docstring se:

# Tool ka description

# Input parameter descriptions

# automatically extract karta hai.

# Agar False ho to docstring ignore kar di jati hai.

@function_tool(use_docstring_info=False)
def get_sum(a: int, b: int):
    """Will be ignored."""
    return a + b


In [ ]:
# 🔹6. failure_error_function: ToolErrorFunction | None = default_tool_error_function
# ✅ Kya karta hai:
# Agar tool fail kare (e.g., invalid input ya crash), to yeh function run hota hai error handle karne ke liye.

def custom_error_handler(error: Exception) -> str:
    return f"Tool failed: {str(error)}"

@function_tool(failure_error_function=custom_error_handler)
def divide(a: int, b: int):
    """Divide two numbers."""
    return a / b


In [ ]:
# 🔹7. strict_mode: bool = True
# ✅ Kya karta hai:
# Agar True ho to input validation strictly hoti hai — agar koi argument missing ya invalid type ho to error throw hoti hai.

# Agar False ho to thoda lax hota hai — default ya None values allow karta hai.

@function_tool(strict_mode=True)
def square(n: int):
    """Returns square of number."""
    return n * n


🔚 Summary Table:
Parameter	        Purpose	             Default

func	The function being wrapped	     Required

name_override	Override tool name	     None

description_override	Override tool description	                           None

docstring_style	Docstring format (google, numpy, etc.)	                                None

use_docstring_info	Use docstring for name/

description	                          True
failure_error_function	Custom error handler	default_tool_error_function
strict_mode	Enforce strict type/input checking	True













 **failure_error_function:** If provided, use this function to generate an error message when
            the tool call fails. The error message is sent to the LLM. If you pass None, then no
            error message will be sent and instead an Exception will be raised.

In [ ]:
from agents import Agent, Runner, function_tool
from typing import Callable
import traceback


# ✅ 1. Custom error function for failed tool calls
def my_error_handler(error: Exception) -> str:
    return f"⚠️ Error occurred: {str(error)}"


# ✅ 2. Tool using all parameters
@function_tool(
    name_override="custom_divide",
    description_override="Divides two numbers with error handling and custom parsing.",
    docstring_style="google",
    use_docstring_info=True,
    failure_error_function=my_error_handler,
    strict_mode=True
)
def divide_numbers(numerator: float, denominator: float) -> float:
    """
    Performs division of two numbers.

    Args:
        numerator: The number to be divided.
        denominator: The number to divide by.

    Returns:
        The result of dividing numerator by denominator.
    """
    return numerator / denominator


# ✅ 3. Define agent that can use this tool
math_agent = Agent(
    name="Math Assistant",
    instructions="You are a helpful math assistant. Use tools to calculate answers accurately.",
    tools=[divide_numbers]  # Tool is already decorated with `function_tool`
)


# ✅ 4. Run the agent using Runner
import asyncio

async def main():
    # 🟢 Case 1: Successful division
    result_ok = await Runner.run(
        math_agent,
        input="Please divide 20 by 5.",
        run_config=config,


    )
    print("✅ Success Output:\n", result_ok.final_output)

    # 🔴 Case 2: Division by zero (handled gracefully by error function)
try:
    result_fail = await Runner.run(
        math_agent,
        input="Please divide 20 by 0.",
        run_config=config,

    )

except Exception as e:
    print("🔴 Error Output:\n", e)


# Run the async main function
asyncio.run(main())


🔴 Error Output:
 Error running tool custom_divide: my_error_handler() takes 1 positional argument but 2 were given
✅ Success Output:
 The answer is 4.0.



def as_tool(

        self,
        
        tool_name: str | None,
        tool_description: str | None,
        custom_output_extractor: Callable[[RunResult], Awaitable[str]] | None = None,
    ) -> Tool:

In [ ]:
from agents import Agent, Runner

# Ek simple agent banate hain
translator_agent = Agent(
    name="TranslatorAgent",
    instructions="You are a translator. You only translate English text to Urdu. Do not say anything else."
)

# Us agent ko ek tool bana dete hain using `.as_tool()`
translator_tool = translator_agent.as_tool(
    tool_name="english_to_urdu_translator",
    tool_description="Translates English sentences into Urdu."
)

# Ab yeh tool kisi aur agent ko diya ja sakta hai
main_agent = Agent(
    name="MainAgent",
    instructions="You are a multi-purpose assistant.",
    tools=[translator_tool]
)

# Run
import asyncio
async def main():
    result = await Runner.run(
        main_agent,
        input="Translate in urdu: 'How are you?'",
        run_config=config,
    )
    print(result.final_output)

asyncio.run(main())


I am sorry, I cannot fulfill this request right now. There seems to be an issue with the API key. Please try again later.



# **stop_at_tool_names**
A list of tool names, any of which will stop the agent from running further.

In [ ]:
stop_at_tool_names: list[str] = ["get_sum"]